In [1]:
import pandas as pd
import numpy as np
from operator import itemgetter
import json
import math

In [2]:
# Reading Dataset and MIF file
def read_data_mif():
    df_obj=pd.read_csv('atm_data.csv')
    ls_obj=df_obj.values.tolist()

    with open('References.json') as output:
        ls_ref = json.load(output)

    with open('MIF.json') as output:
        mif = json.load(output)
    return df_obj, ls_obj, ls_ref, mif

In [3]:
#Calculating Spearman Footrule Distance

def sfd(obj, ref):
    n=len(obj)
    d=0
    for i in range(n):
        d+=abs(obj[i]-ref[i])
    return d

In [4]:
#Calculating query's distance to references

def q_ref_dist(q, ref):
    qtoref_dist=[]
    ordered_qtoref_dist=[]
    for i in range(len(ref)):
        distance=sfd(q,ref[i])
        qtoref_dist.append([i,distance])
    ordered_qtoref_dist=sorted(qtoref_dist, key=itemgetter(1))
    return ordered_qtoref_dist

In [5]:
#Creating Query's PBI
def get_pbi(q, g_ls_reference):
    pbi=[]
    qtoref_distance=[]
    qtoref_distance=q_ref_dist(query, g_ls_reference)
    # print(f'Distance of query to each reference:\n{qtoref_distance}\n')
    for i in range (len(g_ls_reference)):
        pbi.append([qtoref_distance[i][0], i+1])
    pbi=pbi[:10]
    return pbi

In [6]:
def get_close_obj(pbi,mi, g_df_object):
    look_ref=[]
    for i in range(len(pbi)):
        look_ref.append(pbi[i][0])

    coi=[]
    for ref_index in look_ref:
        for i in mi[ref_index][1]:
            if i not in coi:
                coi.append(i[0])

    coi = list(dict.fromkeys(coi))
    df_co=g_df_object.iloc[coi]
    ls_co=df_co.values.tolist()
    return coi, df_co, ls_co

In [7]:
def q_obj_similarity(q, obj):
    qtoobj_dist=[]
    ordered_qtoobj_dist=[]
    for i in range(len(obj)):
        distance=sfd(q,obj[i])
        qtoobj_dist.append([i,distance])
    ordered_qtoobj_dist=sorted(qtoobj_dist, key=itemgetter(1))
    return ordered_qtoobj_dist
    

In [8]:
def print_result(g_similar, g_ls_object):
    class color:
       PURPLE = '\033[95m'
       CYAN = '\033[96m'
       DARKCYAN = '\033[36m'
       BLUE = '\033[94m'
       GREEN = '\033[92m'
       YELLOW = '\033[93m'
       RED = '\033[91m'
       BOLD = '\033[1m'
       UNDERLINE = '\033[4m'
       END = '\033[0m'

    a=query
    for i,element in enumerate(a):
        frac, whole = math.modf(element)
        num=float(a[i])
        (whole, frac) = (int(num), int(str(num)[(len(str(int(num)))+1):]))
        if frac==0:
            a[i]=int(element)
        elif len(str(frac))>6:
            a[i]=round(a[i], 6)
    print(f'{color.BOLD}Query:{color.END}\n\n{(query)}\n')
    print(f'{color.BOLD}20 similar records to the given query:\n{color.END}')
    for  key,value in  enumerate(g_similar):
        a=g_ls_object[value]
        for i,element in enumerate(a):
            frac, whole = math.modf(element)
            num=float(a[i])
            (whole, frac) = (int(num), int(str(num)[(len(str(int(num)))+1):]))
            if frac==0:
                a[i]=int(element)
            elif len(str(frac))>6:
                a[i]=round(a[i], 6)
        print(f'{color.BOLD}{key+1}{color.END}-> record {value}:\n{g_ls_object[value]}\n')

In [9]:
def search(given_query):
    
    df_object, ls_object, ls_reference, metric_inverted= read_data_mif()
    q_pbi=get_pbi(given_query, ls_reference)
    close_objs_indices, df_close_objs, ls_close_objs=get_close_obj(q_pbi, metric_inverted, df_object)
    q_obj_distance=q_obj_similarity(query, ls_close_objs)

    similar=[]
    for i in q_obj_distance[:20]:
        similar.append(close_objs_indices[i[0]])
    
    print_result(similar, ls_object)
    

In [10]:
# Getting query from user
input_query= input('Enter your query (Note: please use comma to seprate the fields):\n') 
input_query_list = list(input_query.split(","))
query=list(np.float_(input_query_list))

# Looking for approximate search of the given query
search(query)

Enter your query (Note: please use comma to seprate the fields):
2018, 11, 23, 5, 21, 1, 47, 0, 24, 20, 157, 1850, 55.675631, 14.536698, 1, 10, 1, 0, 1, 55, 12.53463, 2621942, 9, 279.13, 1008, 87, 5, 110, 1.982272, 0, 321, 1, 1 
Query:

[2018, 11, 23, 5, 21, 1, 47, 0, 24, 20, 157, 1850, 55.675631, 14.536698, 1, 10, 1, 0, 1, 55, 12.53463, 2621942, 9, 279.13, 1008, 87, 5, 110, 1.982272, 0, 321, 1, 1]

20 similar records to the given query:

1-> record 1059566:
[2017, 6, 6, 3, 12, 1, 47, 0, 24, 20, 157, 1850, 55.677431, 12.536698, 1, 10, 1, 0, 1, 55.679379, 12.53463, 2621942, 9, 291.15, 1002, 68, 7, 120, 0.982272, 0, 301, 3, 3]

2-> record 1059736:
[2017, 6, 6, 3, 12, 1, 47, 0, 24, 20, 157, 1850, 55.677431, 12.536698, 1, 9, 1, 0, 1, 55.679379, 12.53463, 2621942, 9, 291.15, 1002, 68, 7, 120, 0.982272, 0, 301, 3, 3]

3-> record 876284:
[2017, 5, 11, 5, 15, 1, 47, 0, 24, 20, 157, 1850, 55.677431, 12.536698, 1, 10, 1, 0, 1, 55.679379, 12.53463, 2621942, 9, 282.35, 1007, 71, 4, 140, 0.982272, 